We are using time-series data found on Kaggle on US housing trends to analyze correlations between average home values, average time on market, and average price cuts. The dataset contains the average home value by city by month from 2000-2024, and from 2018-2024 it includes monthly data about average time on market and price cuts. 

The dataset can be found here, or otherwise in the project file: https://www.kaggle.com/datasets/clovisdalmolinvieira/us-housing-trends-values-time-and-price-cuts

The dataset has a number of missing values, which we handle by dropping rows with consecutive missing values (by category of data, e.g. `HomeValue`, `DaysPending`, etc) or in which the first or last values are missing, and imputing non-consecutive missing data from surrounding values (or in the case of price cuts, setting such a value to 0, or really a tiny epsilon, a somewhat arbitrary decision that can be easily modified). Though this cleaning approach is aggressive and kind of ham-fisted, the dataset is quite large and remains rich even after cleaning it this way.

In [1]:
import pandas as pd
import numpy as np

from statsmodels.tsa.stattools import adfuller

df = pd.read_csv('USRealEstateTrends.csv')

# Cleaning the dataset:
# The dataset includes values for avg home price by city by month 2000-present (5/2024),
# and for 2018-, contains price cuts and avg time on market.
# 
# If a row has multiple consecutive missing values per "category" (column name contains
# `HomePrice`,`CutRaw`,`DaysPending`), or the first or last values are missing, drop the entire row.

epsilon = 1e-10

def clean_data(df):

    categories = ['HomeValue', 'CutRaw', 'DaysPending']

    rows_to_drop = []    # to hold the indices of rows to drop

    for category in categories:
        # get the columns for this category
        category_cols = [col for col in df.columns if category in col]

        # go through each row in the dataframe
        for i in range(len(df)):
            # if the first or last row is missing values, add it to the list of rows to drop
            if i == 0 or i == len(df) - 1:
                if df[category_cols].iloc[i].isnull().any():
                    rows_to_drop.append(i)
                    continue

            # if this row has a missing value, handle it based on the category
            for col in category_cols:
                if pd.isnull(df.loc[i, col]):
                    if category == 'CutRaw':
                        # for 'CutRaw', replace missing values with 0 (really a tiny epsilon = 1^(-10))
                        df.loc[i, col] = epsilon
                    else:
                        # for 'HomeValue' and 'DaysPending', impute or drop based on surrounding values
                        prev_col = category_cols[category_cols.index(col) - 1]
                        next_col = category_cols[(category_cols.index(col) + 1) % len(category_cols)]
                        if not pd.isnull(df.loc[i, prev_col]) and not pd.isnull(df.loc[i, next_col]):
                            # if the previous and next columns have non-missing values, impute the missing value
                            df.loc[i, col] = (df.loc[i, prev_col] + df.loc[i, next_col]) / 2
                        else:
                            # if the previous or next column has a missing value, add the row to the list of rows to drop
                            rows_to_drop.append(i)

    # drop all rows that need to be dropped
    df = df.drop(rows_to_drop)

    return df

df = pd.read_csv('USRealEstateTrends.csv')
df = clean_data(df)
df.to_csv('Cleaned_USRealEstateTrends.csv', index=False)

This time-series data is non-stationary (i.e. does not have a constant mean and variance over time), which we can reasonably infer from the fact that housing prices are affected by trends, boom-bust cycles, seasons, etc. More rigorously, we can use the augmented Dickey-Fuller test to demonstrate with 99% certainty that this time-series data is indeed not stationary. 

In the ADF test, the null hypothesis is that the time-series data is *non-stationary*. If the test statistic is less than the critical value at a certain confidence level, we fail to reject the null hypothesis. Observe that the critical values below are consistently less than the test statistic, implying the data is non-stationary. 

We can also look at the $p$-values: if they are less than, for instance, 0.05, we reject the null hypothesis. Observe that the $p$-values returned are extremely high (~.99), so we fail to reject the null hypothesis and assume with 95% certainty that the data is *non-stationary*.

In [2]:
from statsmodels.tsa.stattools import adfuller

def run_adfuller_test(series):
    """
    Run the Augmented Dickey-Fuller test on a given series.

    Parameters:
    series (pd.Series): The time series data.

    Returns:
    dict: A dictionary containing the test statistic, p-value, number of lags used,
          number of observations, and critical values.
    """

    result = adfuller(series)

    output = {
        'Test Statistic': result[0],
        'p-value': result[1],
        '#Lags Used': result[2],
        '#Observations Used': result[3],
        'Critical Value (1%)': result[4]['1%'],
        'Critical Value (5%)': result[4]['5%'],
        'Critical Value (10%)': result[4]['10%'],
    }

    return output

results = {}
homevalue_columns = [col for col in df.columns if 'HomeValue' in col]
for index, row in df[homevalue_columns].iterrows():
    results[index] = run_adfuller_test(row)
for index, result in results.items():
    print(f"Results for index {index}:")
    print(result)
    print("\n")

Results for index 0:
{'Test Statistic': 0.280808252462644, 'p-value': 0.9764466429592575, '#Lags Used': 15, '#Observations Used': 276, 'Critical Value (1%)': -3.4542672521624214, 'Critical Value (5%)': -2.87206958769775, 'Critical Value (10%)': -2.5723807881747534}


Results for index 1:
{'Test Statistic': -0.6443360615591888, 'p-value': 0.8606459714995169, '#Lags Used': 15, '#Observations Used': 276, 'Critical Value (1%)': -3.4542672521624214, 'Critical Value (5%)': -2.87206958769775, 'Critical Value (10%)': -2.5723807881747534}


Results for index 2:
{'Test Statistic': -0.457980520500763, 'p-value': 0.8999587917954774, '#Lags Used': 14, '#Observations Used': 277, 'Critical Value (1%)': -3.4541800885158525, 'Critical Value (5%)': -2.872031361137725, 'Critical Value (10%)': -2.5723603999791473}


Results for index 3:
{'Test Statistic': -0.6326053975678344, 'p-value': 0.8634430499561223, '#Lags Used': 16, '#Observations Used': 275, 'Critical Value (1%)': -3.454355055831705, 'Critical Va

Since we showed the data is non-stationary, we want to difference it before training and testing a predictive model on it, since time-series forecasting models assume a constant mean and variance in the data.

Trying techniques to make it stationary:
- Natural log of dataset
- STL (Seasonal and Trend decomposition using Loess)
- pandas seasonal differencing

In [3]:
from statsmodels.tsa.seasonal import STL

def apply_stl(df, col):
    stl = STL(df[col], period=12)
    result = stl.fit()
    df[col] = result.resid
for col in df.columns:
    if 'HomeValue' in col or 'CutRaw' in col or 'DaysPending' in col:  # ignore categorical data
        df[col] = df[col].apply(np.log)
        apply_stl(df, col)
        df[col] = df[col].diff(12) # seasonal differencing

df = df.dropna()

df.to_csv('STL_Decomposed_USRealEstateTrends.csv', index=False)

We run ADF again to see whether our data is stationary now. (not yet, but improved)

In [4]:
results = {}
homevalue_columns = [col for col in df.columns if 'HomeValue' in col]
for index, row in df[homevalue_columns].iterrows():
    results[index] = run_adfuller_test(row)
for index, result in results.items():
    print(f"Results for index {index}:")
    print(result)
    print("\n")

Results for index 13:
{'Test Statistic': -3.1339364272942505, 'p-value': 0.02414156519382164, '#Lags Used': 14, '#Observations Used': 277, 'Critical Value (1%)': -3.4541800885158525, 'Critical Value (5%)': -2.872031361137725, 'Critical Value (10%)': -2.5723603999791473}


Results for index 14:
{'Test Statistic': -2.8366186990769062, 'p-value': 0.05323042934605591, '#Lags Used': 8, '#Observations Used': 283, 'Critical Value (1%)': -3.453670163592738, 'Critical Value (5%)': -2.8718077069772248, 'Critical Value (10%)': -2.5722411158835796}


Results for index 15:
{'Test Statistic': -4.073699833483532, 'p-value': 0.0010701469931400905, '#Lags Used': 14, '#Observations Used': 277, 'Critical Value (1%)': -3.4541800885158525, 'Critical Value (5%)': -2.872031361137725, 'Critical Value (10%)': -2.5723603999791473}


Results for index 16:
{'Test Statistic': -1.9080350012061784, 'p-value': 0.3282906851024259, '#Lags Used': 16, '#Observations Used': 275, 'Critical Value (1%)': -3.454355055831705, 